In [1]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pmdarima.arima import auto_arima
from datetime import datetime
import matplotlib.pyplot as plt
import tsfresh
from pmdarima.arima import ADFTest
from tsfresh import extract_features
import joblib
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
import itertools
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [2]:
values = read_csv("../../Time_series_data.csv")

In [3]:
values.head(13)

,Month,Sales
0,2013-01,2815
1,2013-02,2672
2,2013-03,2755
3,2013-04,2721
4,2013-05,2946
5,2013-06,3036
6,2013-07,2282
7,2013-08,2212
8,2013-09,2922
9,2013-10,4301


In [4]:

values = values.rename(columns={values.columns[1]: 'Data'})
values = values.rename(columns={values.columns[0]: 'Time'})
values['Time'] = pd.to_datetime(
    values['Time'], errors='coerce')

In [5]:
freq = pd.infer_freq(values["Time"])

In [6]:
# end_date = pd.to_datetime(x) + pd.DateOffset(months=10)
# print(end_date)

In [7]:
values.set_index('Time', inplace=True)

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(
    values, test_size=0.2, shuffle=False)

In [9]:
model_folder = "Models"
model_filename = f"ses_model.pkl"
model_path = f"{model_folder}/{model_filename}"

In [10]:
import os
os.makedirs(model_folder, exist_ok=True)

In [11]:
def tes_optimizer(train, abg, trend, seasonal,  seasonal_periods, step):
    best_alpha, best_beta, best_gamma, best_mae = None, None, None, float(
        "inf")
    for comb in abg:
        tes_model = ExponentialSmoothing(train, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods).\
            fit(smoothing_level=comb[0], smoothing_slope=comb[1],
                smoothing_seasonal=comb[2])
        y_pred = tes_model.forecast(step)
        mae = mean_absolute_error(test, y_pred)
        if mae < best_mae:
            best_alpha, best_beta, best_gamma, best_mae = comb[0], comb[1], comb[2], mae

    return best_alpha, best_beta, best_gamma, best_mae

In [12]:
def tes_model_tuning(train, test, step, trend, seasonal, seasonal_periods, title="Model Tuning - Triple Exponential Smoothing"):
    alphas = betas = gammas = np.arange(0.10, 1, 0.10)
    abg = list(itertools.product(alphas, betas, gammas))
    best_alpha, best_beta, best_gamma, best_mae = tes_optimizer(
        train, abg=abg, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods, step=step)
    final_model = ExponentialSmoothing(train, trend=trend, seasonal=seasonal).fit(
        smoothing_level=best_alpha, smoothing_slope=best_beta, smoothing_seasonal=best_gamma)
    y_pred = final_model.forecast(step)
    joblib.dump((final_model,best_alpha), model_path)
    mae = mean_absolute_error(test, y_pred)
    mse = mean_squared_error(test, y_pred)
    return y_pred, mae, mse

In [13]:
fcast1, mae, mse = tes_model_tuning(
                train, test, step=test.shape[0], trend='add', seasonal='add', seasonal_periods=12)

c:\Users\haman\anaconda3\envs\doan\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\haman\AppData\Local\Temp\ipykernel_14400\3851052376.py:5: FutureWarning: the 'smoothing_slope'' keyword is deprecated, use 'smoothing_trend' instead
  tes_model = ExponentialSmoothing(train, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods).\
c:\Users\haman\anaconda3\envs\doan\lib\site-packages\statsmodels\tsa\base\tsa_model.py:536: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'
C:\Users\haman\AppData\Local\Temp\ipykernel_14400\3851052376.py:5: FutureWarning: the 'smoothing_slope'' keyword is deprecated, use 'smoothing_trend' instead
  tes_model = ExponentialSmoothing(train, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods).\


In [14]:
tes_model, alpha = joblib.load(model_path)

In [16]:
tes_model

In [ ]:
prediction = pd.DataFrame(fcast1, index=test.index)

prediction.columns = ['Predictions']
prediction.reset_index(inplace=True)

In [ ]:
prediction

In [ ]:
train = train[:80]
train.shape

In [ ]:
arima_model = auto_arima(train, start_p=0, d=1, start_q=0,
                            max_p=2, max_d=2, max_q=2, start_P=0,
                            D=1, start_Q=0, max_P=2, max_D=2,
                            max_Q=2, m=12, seasonal=True,
                            error_action='warn', trace=True,
                            supress_warnings=True, stepwise=True,
                            random_state=20, n_fits=50)

In [ ]:
test

In [ ]:
test1 = test[:12]
test2 = test[12:]
test2

In [ ]:
arima_model.fit(test1)

In [ ]:
n_periods = test2.shape[0]

In [ ]:
prediction = pd.DataFrame(
    arima_model.predict(n_periods=n_periods), index=test2.index)
prediction.columns = ['predicted_values']

In [ ]:
prediction

In [ ]:
n_periods = test.shape[0]

In [ ]:
x = values.index[train.shape[0]]
print(x)

In [ ]:
n_periods = test.shape[0]

In [ ]:
index_future_dates = pd.date_range(
    start=x, periods=n_periods, freq=freq)

index_future_dates

In [ ]:
predicted = model_fit.forecast(n_periods)

In [ ]:
prediction = pd.DataFrame(predicted, index=index_future_dates)
prediction.columns = ['predicted_values']
prediction.reset_index(inplace=True)

In [ ]:
# prediction=prediction.tail(12)
print(prediction)

In [ ]:
prediction.columns.value_counts

In [ ]:
plt.plot(values.index,values['Data'])
plt.plot(index_future_dates,predicted)

In [ ]:
mae = mean_absolute_error(test['Data'], prediction['predicted_values'])
mae

In [ ]:
mape = mean_absolute_percentage_error(test['Data'], prediction['predicted_values'])
mape

In [ ]:
mse = mean_squared_error(test['Data'], prediction['predicted_values'])
mse